In [1]:
'''
Export to Python script: jupyter nbconvert --to script streamAndMeasure.ipynb
'''

import pyaudio
import time
import numpy as np
import queue
import threading
from scipy.signal import find_peaks
import scipy
import pandas as pd
import math
from IPython.display import clear_output

from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import BuPu9, GnBu9, Category20c, RdYlBu, OrRd, RdBu
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, BoxSelectTool, Range1d, Rect, LabelSet, BooleanFilter, BoxAnnotation, DatetimeTickFormatter, CDSView, GroupFilter, NumeralTickFormatter, Label
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

#Plotting options
opts = dict(width=1200, height=600, toolbar_location="above",
      tools='tap, box_zoom, pan, undo, crosshair, reset, wheel_zoom, box_select, save')


Loading BokehJS ...

In [2]:
#https://gist.github.com/Sheljohn/68ca3be74139f66dbc6127784f638920
def cprint( fmt, fg=None, bg=None, style=None ):
    """
    Colour-printer.
        cprint( 'Hello!' )                                  # normal
        cprint( 'Hello!', fg='g' )                          # green
        cprint( 'Hello!', fg='r', bg='w', style='bx' )      # bold red blinking on white
    List of colours (for fg and bg):
        k   black
        r   red
        g   green
        y   yellow
        b   blue
        m   magenta
        c   cyan
        w   white
    List of styles:
        b   bold
        i   italic
        u   underline
        s   strikethrough
        x   blinking
        r   reverse
        y   fast blinking
        f   faint
        h   hide
    """

    COLCODE = {
        'k': 0, # black
        'r': 1, # red
        'g': 2, # green
        'y': 3, # yellow
        'b': 4, # blue
        'm': 5, # magenta
        'c': 6, # cyan
        'w': 7  # white
    }

    FMTCODE = {
        'b': 1, # bold
        'f': 2, # faint
        'i': 3, # italic
        'u': 4, # underline
        'x': 5, # blinking
        'y': 6, # fast blinking
        'r': 7, # reverse
        'h': 8, # hide
        's': 9, # strikethrough
    }

    # properties
    props = []
    if isinstance(style,str):
        props = [ FMTCODE[s] for s in style ]
    if isinstance(fg,str):
        props.append( 30 + COLCODE[fg] )
    if isinstance(bg,str):
        props.append( 40 + COLCODE[bg] )

    # display
    props = ';'.join([ str(x) for x in props ])
    if props:
        return( '\x1b[%sm%s\x1b[0m' % (props, fmt) )
    else:
        return( fmt )

In [3]:

#Wrap into dict to I can pass into thread later if needed
FORMAT = pyaudio.paFloat32
WIDTH = 2
CHANNELS = 1
RATE = 44100 # make this 4khz so I don't need to process later

LENGTH_TO_PROCESS = 12.0 #Seconds of data to buffer before processing

p = pyaudio.PyAudio()

In [4]:

#Create queue to store incoming audio
audioBuffer = queue.Queue()

#Create queue to get processed data
processedData = queue.Queue()
processedDataEvent = threading.Event()
processedDataEvent.clear()

currentTimeLimit = None

#https://pymotw.com/2/threading/
#https://pymotw.com/2/Queue/


#Create thread to process data: Takes audio from buffer, 
#processes it, spits out to processedData queue
def processAudioData(inQueue, outQueue, outEvent, RATE):

    #Collect all audio buffer from queue
    qSize = inQueue.qsize()
    allData = []
    
    for items in range(0, qSize):
        allData.append(inQueue.get())
    
    #Convert to np array so it can be used later on. I'm not sure the first part is needed
    allData = np.array(allData)

    allData = np.frombuffer(allData, dtype=np.float32)
    
    N = len(allData)
    
    #print ("Complete Samplings N", N)
    secs = N / float(RATE)
    Ts = 1.0/RATE # sampling interval in time
    #print ("Timestep between samples Ts", Ts)
    t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
    
    #Put audio into DF
    dfAudio = pd.DataFrame.from_dict( {'amp': allData,
                                       'time': t} )
    #Convert time from floats to time
    dfAudio['time'] = pd.to_timedelta(dfAudio['time'], unit='s')
    dfAudio.set_index('time', inplace=True)

    #Set sampling time. Must be string. #500us->2kHz, 1ms ->1kHz, 2ms->500Hz
    samplingTime = '250us'
    #Manually set sampling frequency - 1/samplingTime
    samplingFrequency = 4000

    #Resample to lower Frequency - Helps with plotting
    dfResampled = dfAudio.resample(samplingTime).sum() 

    #Index must be time
    dfResampled['time'] = dfResampled.index
    
    #Find peaks
    peaksX, _ = find_peaks(dfResampled['amp'], distance=1400)
    
    #Set peaks to time and amp axis
    peaksY = [dfResampled['amp'][j] for j in peaksX]
    peaksX = [dfResampled['time'][j] for j in peaksX]

    dfPeaks = pd.DataFrame.from_dict( {'peaks': peaksY,
                                       'time': peaksX} )
    
    '''
    pltA = figure(**opts, title='Time Series', x_axis_type="datetime")

    #Plot Original
    pltA.line( x=dfResampled['time'], y=dfResampled['amp'], color='black', legend_label='Original Signal', alpha=0.7)

    #Plot Filtered Data
    #pltA.line( x=dfResampled['time'], y=dfResampled['HPF'], color='#6a547d', legend_label='HPF Signal', alpha=0.7)
    #pltA.line( x=dfResampled['time'], y=dfResampled['LPF'], color='#0c6170', legend_label='LPF Signal', alpha=0.7)

    #Plot peaks
    pltA.scatter( x=peaksX, y=peaksY, color=RdBu[3][2], legend_label='peaks', alpha=0.7)

    pltA.legend.click_policy="hide"

    show(pltA)
    '''
    
    #Get time difference between peaks
    peaksDiff = dfPeaks['time'].diff().dropna().tolist()

    #Convert Datetime to Floats
    peaksDiffFloat = []
    for dt in peaksDiff:
        peaksDiffFloat.append( dt.total_seconds() )

    #Show Histogram
    arr_hist, edges = np.histogram(peaksDiffFloat, bins = 12, range=[0.3, 0.7])
    
    #Throw into dict: array, edges, peaksDiff
    
    
    #Spit out processed data - some kind of histogram array
    outQueue.put( (arr_hist, edges) )
    
    processedDataEvent.set()
    

#Call as defined by pyaudio library
def callback(in_data, frame_count, time_info, status):
    
    global currentTimeLimit
    
    #Init varible to track time before processing data
    if currentTimeLimit == None:
        currentTimeLimit = time_info['current_time']
    
    #Push data to queue
    audioBuffer.put(in_data)
    
    #print(time_info)
    
    #Once enough time has elapsed, process it by sending to thread
    if time_info['current_time'] >= currentTimeLimit + LENGTH_TO_PROCESS:
        #Reset time
        currentTimeLimit = time_info['current_time']
        
        #Start thread to process data
        workerThread = threading.Thread(target=processAudioData, args=(audioBuffer, processedData, processedDataEvent, RATE))
        workerThread.setDaemon(True)
        workerThread.start()
        #Maybe add workerThread to list to delete later?
    
    return (in_data, pyaudio.paContinue)


#Show all audio devices
#for i in range(p.get_device_count()):
    #print(p.get_device_info_by_index(i))

In [ ]:
def addColor(stringRow):
    stringLen = len(stringRow)
    
    #Add green
    if stringLen % 2 == 0:
        #Even - Add Green to both middle numbers
        midLow = int((stringLen/2)-1)
        midHigh = int(midLow+1)
        
        stringRow[midLow] = cprint( stringRow[midLow], fg='g', style='f' )
        stringRow[midHigh] = cprint( stringRow[midHigh], fg='g', style='f' )
        
    else:
        #Odd - Add Green to middle number
        mid = int( math.ceil( stringLen/2 ) )
        stringRow[mid] = cprint( stringRow[mid], fg='g', style='f' )
        
    #Add Red
    stringRow[0] = cprint( stringRow[0], fg='r', style='f' )
    stringRow[-1] = cprint( stringRow[-1], fg='r', style='f' )
    
    return stringRow

def formatAndPrint(heading, allReadings, numOfHistoricalReadings):
    #Clear the output on jupyter
    clear_output(wait=True)
    
    #Only keep latest rows per number of histroical readings defines
    if len(allReadings) >= numOfHistoricalReadings:
        allReadings = allReadings[-numOfHistoricalReadings:]
   
    #Format and reprint the heading
    heading = ( ['{:.2f}'.format(x) for x in heading] )
    heading = ( [cprint( x, fg='k', style='b' ) for x in heading] )
    print('\t'.join([str(x) for x in heading]))
    
    actualNumOfReadings = len(allReadings)

    #Process upto the number of historical number of data to display OR number of rows less than number of historical numbers
    i = 0
    while ( (i < numOfHistoricalReadings) and (i < actualNumOfReadings) ) :
        #Convert current row to float
        currentRow = allReadings[i] * 1.0
        
        #Format row
        rowString = ['{:.2f}'.format(x) for x in currentRow]
        coloredString = addColor(rowString)
        
        #print(rowString)
        print('\t'.join([str(x) for x in rowString]))

        i = i + 1
        
    print('\n')
    

stream = p.open(format = FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                output=False,
                stream_callback=callback)

stream.start_stream()


#number of historical data to save
numOfHistoricalReadings = 12

#Heading will always be the same. Get from first reading
heading = []

#Store readings
allReadings = []


try:
    while True:

        time.sleep(0.01)

        #Wait for processed data to be ready
        if processedDataEvent.isSet():
            #Clear flag
            processedDataEvent.clear()
            
            #Get (all) data from queue
            newData = processedData.get()
            
            #Inital heading is blank. It is gotten from the histogram
            if len(heading) < 1:
                heading = newData[1][1:]
                
            #Get latest histogram data
            lastestReading = newData[0]
            
            #Push to array for storing all readings
            allReadings.append(lastestReading)
 
            #Format text and print
            formatAndPrint(heading, allReadings, numOfHistoricalReadings)
            
            #Figure out how to add color - https://stackoverflow.com/questions/287871/how-to-print-colored-text-in-terminal-in-python

except KeyboardInterrupt:
    print('interrupted!')

    stream.stop_stream()
    stream.close()

    p.terminate()

    print('End')

0.33	0.37	0.40	0.43	0.47	0.50	0.53	0.57	0.60	0.63	0.67	0.70
0.00	0.00	4.00	1.00	1.00	1.00	1.00	2.00	2.00	1.00	1.00	0.00


